In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

## Website research and structure of data

In [3]:

base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1']
seasons = ['2014', '2015', '2016', '2017', '2018','2019','2020','2021']

In [4]:

url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")


scripts = soup.find_all('script')


In [5]:
import json

string_with_json_obj = ''


for el in scripts:
    if 'teamsData' in str(el):
      string_with_json_obj = str(el).strip()
      

ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [6]:

data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])


dict_keys(['138', '140', '141', '142', '143', '145', '146', '147', '148', '150', '151', '152', '153', '154', '156', '158', '159', '223', '231', '232'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 3.34997, 'xGA': 2.39239, 'npxG': 3.34997, 'npxGA': 1.64976, 'ppda': {'att': 252, 'def': 19}, 'ppda_allowed': {'att': 263, 'def': 20}, 'deep': 12, 'deep_allowed': 5, 'scored': 4, 'missed': 1, 'xpts': 1.9829999999999999, 'result': 'w', 'date': '2018-08-19 21:15:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 1.70021}


In [7]:

teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

In [8]:

columns = []

values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 3.34997, 2.39239, 3.34997, 1.64976, {'att': 252, 'def': 19}, {'att': 263, 'def': 20}, 12, 5, 4, 1, 1.9829999999999999, 'w', '2018-08-19 21:15:00', 1, 0, 0, 3, 1.70021]


In [9]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))
  
df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181


In [10]:

dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))


Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Eibar.
Added data for Alaves.
Added data for Leganes.
Added data for Girona.
Added data for Real Valladolid.
Added data for SD Huesca.


In [11]:

dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,0,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,0,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178


In [12]:
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  

dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,...,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,13.150000
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,...,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,9.115385


In [13]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [14]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [15]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [16]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [17]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [18]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Barcelona,38,26,9,3,90,36,87,83.28,...,44.93,8.93,43.44,33.14,9.02,16.40,417,171,73.96,-13.04
1,2,Atletico Madrid,38,22,10,6,55,29,76,51.87,...,41.43,12.43,37.72,11.01,11.07,11.10,252,190,59.43,-16.57
2,3,Real Madrid,38,21,5,12,63,46,68,68.65,...,48.68,2.68,42.73,19.24,8.90,14.78,341,168,64.77,-3.23
3,4,Valencia,38,15,16,7,51,35,61,61.88,...,42.85,7.85,36.91,19.66,12.96,9.47,278,215,65.16,4.16
4,5,Sevilla,38,17,8,13,62,47,59,69.16,...,46.71,-0.29,41.51,23.03,10.65,10.02,321,211,65.08,6.08
5,6,Getafe,38,15,14,9,48,35,59,47.03,...,44.23,9.23,39.02,3.56,8.77,5.70,186,196,53.19,-5.81
6,7,Espanyol,38,14,11,13,48,50,53,50.16,...,54.62,4.62,48.55,-1.36,9.86,9.82,241,241,50.09,-2.91
7,8,Athletic Club,38,13,14,11,41,45,53,44.44,...,47.16,2.16,43.44,-4.53,8.30,11.30,221,185,50.01,-2.99
8,9,Real Sociedad,38,13,11,14,45,46,50,47.99,...,48.09,2.09,45.68,-5.13,9.94,9.49,194,208,51.13,1.13
9,10,Alaves,38,13,11,14,39,50,50,40.87,...,54.53,4.53,50.07,-11.43,11.23,7.10,129,270,44.02,-5.98


In [19]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':     position             team  matches  wins  draws  loses  scored  missed  \
0          1        Barcelona       38    26      9      3      90      36   
1          2  Atletico Madrid       38    22     10      6      55      29   
2          3      Real Madrid       38    21      5     12      63      46   
3          4         Valencia       38    15     16      7      51      35   
4          5          Sevilla       38    17      8     13      62      47   
5          6           Getafe       38    15     14      9      48      35   
6          7         Espanyol       38    14     11     13      48      50   
7          8    Athletic Club       38    13     14     11      41      45   
8          9    Real Sociedad       38    13     11     14      45      46   
9         10           Alaves       38    13     11     14      39      50   
10        11       Real Betis       38    14      8     16      44      52   
11        12            Eibar       38    11     14    

In [20]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")


    scripts = soup.find_all('script')
    
    string_with_json_obj = ''


    for el in scripts:
        if 'teamsData' in str(el):
          string_with_json_obj = str(el).strip()

    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')

    data = json.loads(json_data)

    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']

    columns = []

    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      

    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df

    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')

    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data


team  matches  wins  draws  loses  scored  \
             position                                                         
La_liga 2014 1               Barcelona       38    30      4      4     110   
             2             Real Madrid       38    30      2      6     118   
             3         Atletico Madrid       38    23      9      6      67   
             4                Valencia       38    22     11      5      70   
             5                 Sevilla       38    23      7      8      71   
...                                ...      ...   ...    ...    ...     ...   
Ligue_1 2021 16          Clermont Foot       38     9      9     20      38   
             17                Lorient       38     8     12     18      35   
             18          Saint-Etienne       38     7     11     20      42   
             19                   Metz       38     6     13     19      35   
             20               Bordeaux       38     6     13     19      52   

                       missed  pts     xG  xG_diff  ...   xGA  xGA_diff  \
             position                               ...                   
La_liga 2014 1             21   94 102.98    -7.02  ... 28.44      7.44   
             2             38   92  95.77   -22.23  ... 42.61      4.61   
             3             29   78  57.05    -9.95  ... 29.07      0.07   
             4             32   77  55.06   -14.94  ... 39.39      7.39   
             5             45   76  69.53    -1.47  ... 47.86      2.86   
...                       ...  ...    ...      ...  ...   ...       ...   
Ligue_1 2021 16            69   36  42.07     4.07  ... 55.85    -13.15   
             17            63   36  39.32     4.32  ... 54.50     -8.50   
             18            77   32  40.18    -1.82  ... 64.76    -12.24   
             19            69   31  30.15    -4.85  ... 71.65      2.65   
             20            91   31  46.28    -5.72  ... 69.82    -21.18   

                       npxGA  npxGD  ppda_coef  oppda_coef  deep  \
             position                                              
La_liga 2014 1         24.73  73.05       5.68       16.37   489   
             2         38.89  47.21      10.21       12.93   351   
             3         26.84  25.75       8.98        9.24   197   
             4         33.45  16.26       8.71        7.87   203   
             5         41.92  20.18       8.28        9.48   305   
...                      ...    ...        ...         ...   ...   
Ligue_1 2021 16        52.05 -12.39      14.82       11.75   159   
             17        49.94 -12.90      15.65       10.54   163   
             18        58.68 -22.30      12.01       10.96   146   
             19        67.85 -38.46      15.37       10.14    84   
             20        62.84 -19.59      12.45       11.77   173   

                       deep_allowed  xpts  xpts_diff  
             position                                 
La_liga 2014 1                  114 94.08       0.08  
             2                  153 81.75     -10.25  
             3                  123 73.14      -4.86  
             4                  172 63.71     -13.29  
             5                  168 67.39      -8.61  
...                             ...   ...        ...  
Ligue_1 2021 16                 212 43.38       7.38  
             17                 221 40.73       4.73  
             18                 196 39.69       7.69  
             19                 265 26.71      -4.29  
             20                 238 38.21       7.21  

[784 rows x 21 columns]

In [28]:
data.to_csv('teams.csv')